In [1]:
from datetime import datetime,timedelta
from scipy.stats import iqr,skew,kurtosis,mode
from joblib import Parallel,delayed
import zipfile
import shutil
from sklearn.feature_selection import SelectKBest,f_classif
from sklearn.decomposition import PCA
from pprint import pprint
from sklearn.metrics import f1_score,r2_score,classification_report
from sklearn.model_selection import ParameterGrid
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier,RandomForestRegressor
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.metrics import confusion_matrix,f1_score,precision_score,recall_score,accuracy_score
import itertools
from sklearn.model_selection import ParameterGrid, cross_val_predict, GroupKFold,GridSearchCV,StratifiedKFold
from sklearn import preprocessing
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from collections import Counter
from sklearn import preprocessing
import tensorflow as tf
import numpy as np
seed = 100
tf.random.set_seed(seed)
np.random.seed(seed)
import os
import pandas as pd
import pickle
from tensorflow import keras
from tensorflow.keras import backend as K
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping
from sklearn.model_selection import train_test_split,LeavePGroupsOut
from keras.backend import expand_dims, repeat_elements
from keras.layers import Lambda
from tensorflow.keras.layers import Conv1D,BatchNormalization,Dropout,InputLayer,MaxPooling1D,Flatten,RepeatVector,Dense,Input,Activation,GRU,Bidirectional,LSTM
from tensorflow.keras.models import Model, Sequential
from sklearn.metrics import accuracy_score
import tensorflow_addons as tfa
import warnings
warnings.filterwarnings('ignore')
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only use the first GPU
    try:
        tf.config.experimental.set_visible_devices(gpus[1], 'GPU')
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
    except RuntimeError as e:
        # Visible devices must be set before GPUs have been initialized
        print(e)

/home/azim/miniconda3/envs/test1/lib/python3.8/site-packages/tensorflow_addons/utils/ensure_tf_install.py:38: UserWarning: You are currently using a nightly version of TensorFlow (2.5.0-dev20210310). 
TensorFlow Addons offers no support for the nightly versions of TensorFlow. Some things might work, some other might not. 
If you encounter a bug, do not file an issue on GitHub.
  warnings.warn(


4 Physical GPUs, 1 Logical GPU


In [2]:
filepath_file = './data/episode_encoded_lagged_data/episode_encoded_'+'lagged_{}_windows_standardized_phenotype_no_episode_cluster_check'
def get_X_y_groups(n_lag=10):
    data = pickle.load(open(filepath_file.format(n_lag),'rb'))

    X_feature = np.concatenate(data.feature_final.values)
    X_static =  np.concatenate(data.static_features.values)

    X_stress_episode = np.concatenate(data.stress_episode.values)
    X_quit_episode = np.concatenate(data.quit_episode.values)
    X_activity_episode = np.concatenate(data.activity_episode.values)
    X_smoking_episode = np.concatenate(data.smoking_episode.values)

    y_time = data['time'].values
    y = data['label'].values
    groups = data['user'].values
    
    return X_feature,X_static,X_stress_episode,X_quit_episode,X_activity_episode,X_smoking_episode,y_time,y,groups


def get_train_test_indexes(groups,n_groups_split = 10,n_val_groups = 10):
    groups_unique = np.unique(groups)
    groups_split = np.array_split(groups_unique,n_groups_split)
    indexes = []
    for this_groups in groups_split:
        train_groups = np.array([a for a in groups_unique if a not in this_groups])
        val_groups = np.random.choice(train_groups,n_val_groups)
        train_groups = np.array([a for a in groups_unique if a not in list(this_groups)+list(val_groups)])
        test_groups = this_groups
        train_index,test_index = np.array([i for i,a in enumerate(groups) 
                                           if a in train_groups]),np.array([i for i,a in enumerate(groups) 
                                                                               if a in test_groups])
        val_index = np.array([i for i,a in enumerate(groups) 
                                           if a in val_groups])
        indexes.append([train_index,test_index,val_index])
    return indexes


from sklearn import metrics
def f1Bias_scorer_CV(probs, y, ret_bias=True):
    precision, recall, thresholds = metrics.precision_recall_curve(y, probs)

    f1,bias = 0.0,.5
    min_recall = .7
    for i in range(0, len(thresholds)):
        if not (precision[i] == 0 and recall[i] == 0) and recall[i]>min_recall:
            f = 2 * (precision[i] * recall[i]) / (precision[i] + recall[i])
            if f > f1:
                f1 = f
                bias = thresholds[i]

    if ret_bias:
        return f1, bias
    else:
        return f1


def get_model():
    n_t,n_f = train_feature.shape[1],train_feature.shape[2]
    print(n_t,n_f)
    x_input = Input(shape=(n_t,n_f))
    # x_feature = Conv1D(100,1,activation='linear')(x_input)
    x_feature = Conv1D(100,1,activation='tanh')(x_input)
    x_feature = Dropout(.2)(x_feature)
    x_feature = LSTM(20,activation='tanh',return_sequences=False)(x_feature)
    x_feature = Dropout(.3)(x_feature)
    x_feature = Flatten()(x_feature)
    x_feature = Dense(10,activation='relu')(x_feature)
    # x_final = Dense(1,activation='sigmoid')(x_feature)

    n_sf = train_static.shape[1]
    x_input_static = Input(shape=(n_sf))
    x_static = Dense(10,activation='relu')(x_input_static)
    x_static = Dense(10,activation='relu')(x_static)
    n_timesteps = train_stress.shape[-2]
    n_episodes_stress,n_episodes_quit,n_episodes_activity,n_episodes_smoking = train_stress.shape[1],train_quit.shape[1],train_activity.shape[1],train_smoking.shape[1]
    x_alpha_stress = Dense(1,activation='sigmoid',name='alpha_stress')(x_static)
    x_alpha_stress = RepeatVector(n_timesteps)(x_alpha_stress)
    x_alpha_stress = Lambda(lambda x: repeat_elements(expand_dims(x, axis=1), n_episodes_stress, 1))(x_alpha_stress)
    x_alpha_quit = Dense(1,activation='sigmoid',name='alpha_quit')(x_static)
    x_alpha_quit = RepeatVector(n_timesteps)(x_alpha_quit)
    x_alpha_quit = Lambda(lambda x: repeat_elements(expand_dims(x, axis=1), n_episodes_quit, 1))(x_alpha_quit)
    x_alpha_activity = Dense(1,activation='sigmoid',name='alpha_activity')(x_static)
    x_alpha_activity = RepeatVector(n_timesteps)(x_alpha_activity)
    x_alpha_activity = Lambda(lambda x: repeat_elements(expand_dims(x, axis=1), n_episodes_activity, 1))(x_alpha_activity)
    x_alpha_smoking = Dense(1,activation='sigmoid',name='alpha_smoking')(x_static)
    x_alpha_smoking = RepeatVector(n_timesteps)(x_alpha_smoking)
    x_alpha_smoking = Lambda(lambda x: repeat_elements(expand_dims(x, axis=1), n_episodes_smoking, 1))(x_alpha_smoking)

    n_dim = 3
    x_stress = Input(shape=(n_episodes_stress,n_timesteps,n_dim))
    stress_alpha_time = tf.math.multiply(x_alpha_stress[:,:,:,0]*-1,x_stress[:,:,:,0])
    stress_alpha_time_exp = tf.math.exp(stress_alpha_time)

    x_stress_amplitude = x_stress[:,:,:,1]
    stress_amplitude_coeff = Dense(1,activation='relu',name='amplitude_stress')(x_static)
    stress_amplitude_coeff = Lambda(lambda x: expand_dims(x, axis=2))(stress_amplitude_coeff)
    print(stress_amplitude_coeff.shape,'coeff',x_stress_amplitude.shape,'amplitude')
    x_stress_amplitude = tf.math.multiply(x_stress_amplitude,stress_amplitude_coeff)
#     print(x_stress_amplitude.shape,'final')
    
    x_stress_duration = x_stress[:,:,:,2]
    stress_duration_coeff = Dense(1,activation='sigmoid',name='duration_stress')(x_static)
    stress_duration_coeff = Lambda(lambda x: expand_dims(x, axis=2))(stress_duration_coeff)
    x_stress_duration = tf.math.multiply(x_stress_duration,stress_duration_coeff)
    
    x_stress_all = tf.math.add(x_stress_amplitude,x_stress_duration)
    
#     print(x_stress_all.shape,stress_alpha_time_exp.shape)
    stress_alpha_time_exp_amplitude = tf.math.multiply(stress_alpha_time_exp,x_stress_all)
    
#     print(stress_alpha_time_exp_amplitude.shape)
    stress_final = tf.math.reduce_sum(stress_alpha_time_exp_amplitude,axis=1)
#     print(stress_final.shape)
    stress_final = Lambda(lambda x: expand_dims(x, axis=2))(stress_final)
#     print(stress_final.shape)
    # stress_final = LSTM(10,activation='tanh',return_sequences=True)(stress_final)
    
    
    
    x_quit = Input(shape=(n_episodes_quit,n_timesteps,n_dim))
#     quit_alpha_time = tf.math.multiply(x_alpha_quit[:,:,:,0]*-1,x_quit[:,:,:,0])
#     quit_alpha_time_exp = tf.math.exp(quit_alpha_time)
#     quit_alpha_time_exp_amplitude = tf.math.multiply(quit_alpha_time_exp,x_quit[:,:,:,1])
#     quit_final = tf.math.reduce_sum(quit_alpha_time_exp_amplitude,axis=1)
#     quit_final = Lambda(lambda x: expand_dims(x, axis=2))(quit_final)
#     quit_final = LSTM(5,activation='tanh',return_sequences=True)(quit_final)

    
    
    x_smoking = Input(shape=(n_episodes_smoking,n_timesteps,n_dim))
    smoking_alpha_time = tf.math.multiply(x_alpha_smoking[:,:,:,0]*-1,x_smoking[:,:,:,0])
    smoking_alpha_time_exp = tf.math.exp(smoking_alpha_time)
    
    x_smoking_amplitude = x_smoking[:,:,:,1]
    smoking_amplitude_coeff = Dense(1,activation='sigmoid',name='amplitude_smoking')(x_static)
    smoking_amplitude_coeff = Lambda(lambda x: expand_dims(x, axis=2))(smoking_amplitude_coeff)
    x_smoking_amplitude = tf.math.multiply(x_smoking_amplitude,smoking_amplitude_coeff)
    
    x_smoking_duration = x_smoking[:,:,:,2]
    smoking_duration_coeff = Dense(1,activation='sigmoid',name='duration_smoking')(x_static)
    smoking_duration_coeff = Lambda(lambda x: expand_dims(x, axis=2))(smoking_duration_coeff)
    x_smoking_duration = tf.math.multiply(x_smoking_duration,smoking_duration_coeff)
    
    x_smoking_all = tf.math.add(x_smoking_amplitude,x_smoking_duration)
    smoking_alpha_time_exp_amplitude = tf.math.multiply(smoking_alpha_time_exp,x_smoking_all)
    smoking_final = tf.math.reduce_sum(smoking_alpha_time_exp_amplitude,axis=1)
    smoking_final = Lambda(lambda x: expand_dims(x, axis=2))(smoking_final)
    # smoking_final = LSTM(10,activation='tanh',return_sequences=True)(smoking_final)

    
    
    x_activity = Input(shape=(n_episodes_activity,n_timesteps,n_dim))
    activity_alpha_time = tf.math.multiply(x_alpha_activity[:,:,:,0]*-1,x_activity[:,:,:,0])
    activity_alpha_time_exp = tf.math.exp(activity_alpha_time)
    
    x_activity_amplitude = x_activity[:,:,:,1]
    activity_amplitude_coeff = Dense(1,activation='sigmoid',name='amplitude_activity')(x_static)
    activity_amplitude_coeff = Lambda(lambda x: expand_dims(x, axis=2))(activity_amplitude_coeff)
    x_activity_amplitude = tf.math.multiply(x_activity_amplitude,activity_amplitude_coeff)
    
    x_activity_duration = x_activity[:,:,:,2]
    activity_duration_coeff = Dense(1,activation='sigmoid',name='duration_activity')(x_static)
    activity_duration_coeff = Lambda(lambda x: expand_dims(x, axis=2))(activity_duration_coeff)
#     print(activity_duration_coeff)
    x_activity_duration = tf.math.multiply(x_activity_duration,activity_duration_coeff)
    
    x_activity_all = tf.math.add(x_activity_amplitude,x_activity_duration)
    activity_alpha_time_exp_amplitude = tf.math.multiply(activity_alpha_time_exp,x_activity_all)
    activity_final = tf.math.reduce_sum(activity_alpha_time_exp_amplitude,axis=1)
    activity_final = Lambda(lambda x: expand_dims(x, axis=2))(activity_final)
    # activity_final = LSTM(10,activation='tanh',return_sequences=True)(activity_final)
    
    
    x_episode = tf.concat([activity_final, stress_final, smoking_final],2)
    # x_episode = Conv1D(100,10,activation='relu')(x_episode)
    # x_episode = Conv1D(100,10,activation='tanh')(x_episode)
    # x_episode = LSTM(100,activation='tanh',return_sequences=True)(x_episode)
    x_episode = LSTM(20,activation='tanh',return_sequences=False)(x_episode)
    x_episode = Dropout(.3)(x_episode)
    x_episode = Flatten()(x_episode)
    x_episode = Dense(10,activation='relu')(x_episode)

    # merged = tf.concat([x_feature,x_episode],1)
    merged = x_feature
    merged = Dense(10,activation='relu')(merged)
    # merged = Dense(10,activation='relu')(merged)
    merged = Lambda(lambda x: K.l2_normalize(x,axis=1),name='normalize')(merged)
    output = Dense(2,activation='relu')(merged)
    output = Dense(2,activation='softmax',name='softmax')(output)
    # output = Dense(2,activation='softmax')(merged)
    # output = Reshape((-1,1))(output)
#     output = Activation('softmax',name='softmax')(output)
    model = Model(inputs=[x_input,x_input_static,x_stress,x_activity,x_smoking,x_quit], outputs=[output,merged])
    model.compile(
        loss={'softmax':tf.losses.SparseCategoricalCrossentropy(),'normalize':tfa.losses.TripletSemiHardLoss()},
        metrics={'softmax':['acc']},
        optimizer='adam'
    )
    return model
# tf.keras.losses.SparseCategoricalCrossentropy()
# from keras.layers import Reshape
def myloss(alpha=.2):
    
    def custom_loss(true,pred):
        m = tf.keras.losses.BinaryCrossentropy()
        return f1_weighted(true,pred)+m(true,pred)*alpha
    
    
    def f1_weighted(true, pred): #shapes (batch, 4)

        #for metrics include these two lines, for loss, don't include them
        #these are meant to round 'pred' to exactly zeros and ones
        #predLabels = K.argmax(pred, axis=-1)
        #pred = K.one_hot(predLabels, 4) 


        ground_positives = K.sum(true, axis=0) + K.epsilon()       # = TP + FN
        pred_positives = K.sum(pred, axis=0) + K.epsilon()         # = TP + FP
        true_positives = K.sum(true * pred, axis=0) + K.epsilon()  # = TP
            #all with shape (4,)

        precision = true_positives / pred_positives 
        recall = true_positives / ground_positives
            #both = 1 if ground_positives == 0 or pred_positives == 0
            #shape (4,)

        f1 = 2 * (precision * recall) / (precision + recall + K.epsilon())
            #still with shape (4,)

        weighted_f1 = f1 * ground_positives / K.sum(ground_positives) 
        weighted_f1 = K.sum(weighted_f1)


        return 1 - weighted_f1
    return custom_loss


In [3]:
from sklearn.preprocessing import OneHotEncoder
def get_X_y_groups(n_lag=10):
    data = pickle.load(open(filepath_file.format(n_lag),'rb'))

    X_feature = np.concatenate(data.feature_final.values)
    X_static =  np.concatenate(data.static_features.values)

    X_stress_episode = np.concatenate(data.stress_episode.values)
    X_quit_episode = np.concatenate(data.quit_episode.values)
    X_activity_episode = np.concatenate(data.activity_episode.values)
    X_smoking_episode = np.concatenate(data.smoking_episode.values)

    y_time = data['time'].values
    print(data['label'].unique())
    # y =  OneHotEncoder().fit_transform(data['label'].values.reshape(-1,1)).todense()
    y = data['label'].values
    groups = data['user'].values
    
    return X_feature,X_static,X_stress_episode,X_quit_episode,X_activity_episode,X_smoking_episode,y_time,y,groups


from sklearn.metrics import roc_auc_score
use_standardization = True
for n_lag in [15]:
    all_data = []
    columns = ['alpha_stress','alpha_smoking','alpha_activity']
    amplitude_duration_data = []
    amplitude_duration_columns = ['amplitude_stress','duration_stress'] 
    data_cluster = pickle.load(open(filepath_file.format(n_lag),'rb'))
    temp = data_cluster.groupby(['user','cluster_label']).count().index.values
    users = np.array([a[0] for a in temp])
    labels = np.array([a[1] for a in temp])
    cluster_dict = {}
    for i,a in enumerate(users):
        cluster_dict[a] = labels[i]
    n_groups = len(np.unique(users))
    # n_groups = 20
    X_feature,X_static,X_stress_episode,X_quit_episode,X_activity_episode,X_smoking_episode,y_time,y,groups = get_X_y_groups(n_lag)
    # y[y<0] = 0
    y = np.float32(y)
    indexes = get_train_test_indexes(groups,n_groups_split = 5,n_val_groups=5)
    final_y_time = []
    final_probs = []
    final_y = []
    final_groups = []
    bias_dict = {}
    val_results = {}
    for kk,yyyy in enumerate(indexes):
        train_index,test_index,val_index = yyyy
        
        X_feature_train,X_feature_test = X_feature[train_index],X_feature[test_index]
        X_static_train,X_static_test = X_static[train_index],X_static[test_index]
        X_stress_episode_train,X_stress_episode_test = X_stress_episode[train_index], X_stress_episode[test_index]
        X_quit_episode_train,X_quit_episode_test = X_quit_episode[train_index], X_quit_episode[test_index]
        X_activity_episode_train,X_activity_episode_test = X_activity_episode[train_index], X_activity_episode[test_index]
        X_smoking_episode_train,X_smoking_episode_test = X_smoking_episode[train_index], X_smoking_episode[test_index]
        y_train,y_test,groups_train,groups_test,time_train,time_test = y[train_index],y[test_index],groups[train_index],groups[test_index],y_time[train_index],y_time[test_index]
        
        X_feature_val,X_static_val,X_stress_episode_val,X_quit_episode_val,\
        X_activity_episode_val,X_smoking_episode_val,y_val,groups_val,time_val = X_feature[val_index],X_static[val_index],X_stress_episode[val_index],X_quit_episode[val_index],\
                                                                                X_activity_episode[val_index],X_smoking_episode[val_index],y[val_index],groups[val_index],y_time[val_index]
        
        # y_train = np.array(y_train).reshape(len(y_train),-1)
        positive_train_index = np.where(y_train==1)[0]
        negative_train_index = np.where(y_train==0)[0]
        len_positive = len(positive_train_index)
        n_iters = 5
        test_preds = []
        bias_pred = []
        for i,n_iter in enumerate(range(n_iters)):
            np.random.seed(np.random.randint(109))
            indexes_sampled = np.array(list(positive_train_index)*5+list(np.random.choice(negative_train_index,len_positive*5,replace=False)))
            train_feature = X_feature_train[indexes_sampled]
            train_static = X_static_train[indexes_sampled]
            train_stress = X_stress_episode_train[indexes_sampled]
            train_quit = X_quit_episode_train[indexes_sampled]
            train_activity = X_activity_episode_train[indexes_sampled]
            train_smoking = X_smoking_episode_train[indexes_sampled]
            train_y = y_train[indexes_sampled]
            from keras import backend as K
            K.clear_session()
            model = get_model()
            # model.summary()

            filepath = './models/lag_'+str(n_lag)+'_iter_'+str(n_iter)+'_split_episode_model_v9_new_validation_set_include_smoking_focal_loss_static_coeff_phenotype_lower_model_v5_triplet_no_episode.hdf5'
            checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=0, save_best_only=True, mode='min',save_weights_only=False)
            es = EarlyStopping(monitor='val_loss', mode='min', verbose=1,patience=60)
            callbacks_list = [es,checkpoint]
            train_feature,val_feature,train_static,val_static,train_stress,val_stress, \
            train_smoking,val_smoking,train_quit,val_quit,train_activity,val_activity, \
            train_y,val_y = train_test_split(
                                            train_feature,
                                            train_static,
                                            train_stress,
                                            train_smoking,
                                            train_quit,
                                            train_activity,
                                            train_y,
                                            test_size=.15,
                                            stratify=train_y
                                            )
            val_feature = np.concatenate([val_feature,X_feature_val])
            val_static = np.concatenate([val_static,X_static_val])
            val_stress = np.concatenate([val_stress,X_stress_episode_val])
            val_activity = np.concatenate([val_activity,X_activity_episode_val])
            val_smoking = np.concatenate([val_smoking,X_smoking_episode_val])
            val_quit = np.concatenate([val_quit,X_quit_episode_val])
            val_y = np.array(list(val_y)+list(y_val))
            
            model.fit([train_feature,train_static,train_stress,train_activity,train_smoking,train_quit],train_y,
            validation_data=([val_feature,val_static,val_stress,val_activity,val_smoking,val_quit],val_y), epochs=300, batch_size=200,
                        verbose=0,callbacks=callbacks_list,shuffle=True)
            if os.path.isfile(filepath):
                model.load_weights(filepath)
            # temp_model = Model(model.input[1],[model.get_layer(c).output for c in columns+amplitude_duration_columns])
            # X_static_test_new = np.hstack((X_static_test, np.zeros((X_static_test.shape[0], 3), dtype=X_static_test.dtype)))
            # X_static_test_new[:,-3] = np.mean(X_stress_episode_test[:,:,0,1],axis=1)
            # X_static_test_new[:,-2] = np.mean(X_stress_episode_test[:,:,0,2],axis=1)
            # X_static_test_new[:,-1] = [cluster_dict[a] for a in groups_test]
            # X_static_test_new = np.unique(X_static_test_new,axis=0)
            # pred_list = list(zip(*[list(a.reshape(-1)) for a in temp_model.predict(X_static_test_new[:,:-3])]))
            # for ii,a in enumerate(pred_list):
            #     for jj,c_name in enumerate(columns+amplitude_duration_columns):
            #         all_data.append([a[jj],
            #                         c_name,
            #                         X_static_test_new[ii][-1],
            #                         a[-2]*X_static_test_new[ii][-3],
            #                         a[-1]*X_static_test_new[ii][-2],
            #                         n_lag,
            #                         kk,
            #                         i])

            test_preds.append(model.predict([X_feature_test,X_static_test,X_stress_episode_test,
                                        X_activity_episode_test,X_smoking_episode_test,X_quit_episode_test])[0][:,1])
            bias_pred.append(model.predict([X_feature_val,X_static_val,X_stress_episode_val,X_activity_episode_val,
                                            X_smoking_episode_val,X_quit_episode_val])[0][:,1])
            
        from sklearn.preprocessing import MinMaxScaler
        y_test_pred = np.concatenate([a.reshape(-1,1) for a in test_preds],axis=1)
        bias_pred = np.concatenate([a.reshape(-1,1) for a in bias_pred],axis=1)
        # print(roc_auc_score(y_test,y_test_pred))
        final_y_time.extend(list(time_test))
        final_probs.extend(list(y_test_pred))
        final_y.extend(list(y_test))
        final_groups.extend(list(groups_test))
        for group_b in np.unique(groups_test):
            bias_dict[group_b] = []
            for kkk in range(bias_pred.shape[1]):
                f1,bias = f1Bias_scorer_CV(bias_pred[:,kkk].reshape(-1),np.int64(y_val.reshape(-1)))
                f1_test, bias_test = f1Bias_scorer_CV(y_test_pred[:,kkk],np.int64(y_test.reshape(-1)))
                
                print(group_b,
                'val results',
                f1,bias,
                'index',kkk,
                'test results',
                f1_test,bias_test,
                'test result with val bias',
                f1_score(y_test,np.array(y_test_pred[:,kkk]>bias,dtype=np.int64)))
                
                bias_dict[group_b].append(bias)
            val_results[group_b] = [time_val,bias_pred,y_val,groups_val]
        print(len(np.unique(final_groups)))
        # print(bias_dict)
    final_y_time,final_probs,final_y,final_groups = np.array(final_y_time),np.array(final_probs),np.array(final_y),np.array(final_groups)
    pickle.dump([final_y_time,final_probs,final_y,final_groups,bias_dict,val_results],open('./data/output_final/result_no_episode_new_lag_{}_triplet_loss.p'.format(n_lag),'wb'))

[0 1]
15 70
(None, 1, 1) coeff (None, 10, 30) amplitude
Epoch 00112: early stopping
15 70
(None, 1, 1) coeff (None, 10, 30) amplitude
Epoch 00149: early stopping
15 70
(None, 1, 1) coeff (None, 10, 30) amplitude
Epoch 00112: early stopping
15 70
(None, 1, 1) coeff (None, 10, 30) amplitude
Epoch 00118: early stopping
15 70
(None, 1, 1) coeff (None, 10, 30) amplitude
Epoch 00195: early stopping
3002 val results 0.1879895561357702 0.003995239 index 0 test results 0.22380643767552386 0.0038435208 test result with val bias 0.2234209934816813
3002 val results 0.3159268929503916 0.0044131144 index 1 test results 0.24830032515518766 0.0044055325 test result with val bias 0.2190709046454768
3002 val results 0.23330365093499555 0.005212347 index 2 test results 0.23191433794724473 0.005206288 test result with val bias 0.22470030666294954
3002 val results 0.24113475177304963 0.0045443163 index 3 test results 0.2478247824782478 0.004535908 test result with val bias 0.23672131147540984
3002 val resu

In [ ]:
len